In [ ]:
import pandas as pd
import numpy as np

In [ ]:
#pip install -U scikit-learn

In [ ]:
import sklearn as sk

In [ ]:
#reading the file into a dataframe
import io
from google.colab import files
uploaded = files.upload()
credits_df = pd.read_csv(io.BytesIO(uploaded['tmdb_5000_credits.csv']))
uploaded = files.upload()
movies_df = pd.read_csv(io.BytesIO(uploaded['tmdb_5000_movies.csv']))

Saving tmdb_5000_credits.csv to tmdb_5000_credits.csv


Saving tmdb_5000_movies.csv to tmdb_5000_movies.csv


In [ ]:
#merging both dataframes
movies_df.genres
credits_df.columns = ['id','title','cast','crew']
movies_df = movies_df.merge(credits_df, on = 'id')
movies_df.cast

0       [{"cast_id": 242, "character": "Jake Sully", "...
1       [{"cast_id": 4, "character": "Captain Jack Spa...
2       [{"cast_id": 1, "character": "James Bond", "cr...
3       [{"cast_id": 2, "character": "Bruce Wayne / Ba...
4       [{"cast_id": 5, "character": "John Carter", "c...
                              ...                        
4798    [{"cast_id": 1, "character": "El Mariachi", "c...
4799    [{"cast_id": 1, "character": "Buzzy", "credit_...
4800    [{"cast_id": 8, "character": "Oliver O\u2019To...
4801    [{"cast_id": 3, "character": "Sam", "credit_id...
4802    [{"cast_id": 3, "character": "Herself", "credi...
Name: cast, Length: 4803, dtype: object

In [ ]:
# the features that we will consider are cast , crew , keywords and genres
from ast import literal_eval
features = ['cast','crew','keywords','genres']
for feature in features:
  movies_df[feature] = movies_df[feature].apply(literal_eval)
movies_df.cast

0       [{'cast_id': 242, 'character': 'Jake Sully', '...
1       [{'cast_id': 4, 'character': 'Captain Jack Spa...
2       [{'cast_id': 1, 'character': 'James Bond', 'cr...
3       [{'cast_id': 2, 'character': 'Bruce Wayne / Ba...
4       [{'cast_id': 5, 'character': 'John Carter', 'c...
                              ...                        
4798    [{'cast_id': 1, 'character': 'El Mariachi', 'c...
4799    [{'cast_id': 1, 'character': 'Buzzy', 'credit_...
4800    [{'cast_id': 8, 'character': 'Oliver O’Toole',...
4801    [{'cast_id': 3, 'character': 'Sam', 'credit_id...
4802    [{'cast_id': 3, 'character': 'Herself', 'credi...
Name: cast, Length: 4803, dtype: object

In [ ]:
def get_director(x):
    for i in x:
        if i["job"] == "Director":
            return i["name"]
    return np.nan


def get_list(x):
    if isinstance(x, list):
         #print(type(i['name']) for i in x)
         names = [i['name'] for i in x]
         #print(names)
         if len(names) > 3:
             names = names[:3]
         return names
    return []


In [ ]:
movies_df["director"] = movies_df["crew"].apply(get_director)

features = ["cast", "keywords","genres"]
for feature in features:
    movies_df[feature] = movies_df[feature].apply(get_list)

######################################################################
# ######################## TO DO ########################################
# Insure that the script can be run without having to check if parts have been run before
# => the script should not be replacing original columns and we need to create new dummy columns to hold older data

In [ ]:
movies_df[['title_x', 'cast', 'director', 'keywords', 'genres']].head()

,title_x,cast,director,keywords,genres
0,Avatar,"[Sam Worthington, Zoe Saldana, Sigourney Weaver]",James Cameron,"[culture clash, future, space war]","[Action, Adventure, Fantasy]"
1,Pirates of the Caribbean: At World's End,"[Johnny Depp, Orlando Bloom, Keira Knightley]",Gore Verbinski,"[ocean, drug abuse, exotic island]","[Adventure, Fantasy, Action]"
2,Spectre,"[Daniel Craig, Christoph Waltz, Léa Seydoux]",Sam Mendes,"[spy, based on novel, secret agent]","[Action, Adventure, Crime]"
3,The Dark Knight Rises,"[Christian Bale, Michael Caine, Gary Oldman]",Christopher Nolan,"[dc comics, crime fighter, terrorist]","[Action, Crime, Drama]"
4,John Carter,"[Taylor Kitsch, Lynn Collins, Samantha Morton]",Andrew Stanton,"[based on novel, mars, medallion]","[Action, Adventure, Science Fiction]"


In [ ]:
# text formatting and cleaning
#cleans strings and list -> lower case letters and reducing blank space
def clean_row(x):
  if(isinstance(x,list)):
    return [str.lower(i.replace(" ",'')) for i in x]
  elif(isinstance(x,str)):
    return str.lower(x.replace(" ",''))
  else:
    return ""


In [ ]:
features = ['cast', 'keywords', 'director', 'genres']
for feature in features:
    movies_df[feature] = movies_df[feature].apply(clean_row)

In [ ]:
#merging text features into a single feature
def merge_features(x):
  return ' '.join(x['cast'])+' '.join(x['keywords'])+' '.join(x['director'])+' '.join(x['genres'])

In [ ]:
movies_df['merged_feature'] = movies_df.apply(merge_features,axis=1)
movies_df.merged_feature

0       samworthington zoesaldana sigourneyweavercultu...
1       johnnydepp orlandobloom keiraknightleyocean dr...
2       danielcraig christophwaltz léaseydouxspy based...
3       christianbale michaelcaine garyoldmandccomics ...
4       taylorkitsch lynncollins samanthamortonbasedon...
                              ...                        
4798    carlosgallardo jaimedehoyos petermarquardtunit...
4799    edwardburns kerrybishé marshadietleine d w a r...
4800    ericmabius kristinbooth crystallowedate loveat...
4801    danielhenney elizacoupe billpaxtond a n i e l ...
4802    drewbarrymore brianherzlinger coreyfeldmanobse...
Name: merged_feature, Length: 4803, dtype: object

In [ ]:
#transforming merged feature into a vector and calculating cosine similarity
#resetting indices
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

count_vectorizer = CountVectorizer(stop_words="english")
count_matrix = count_vectorizer.fit_transform(movies_df["merged_feature"])

print(count_matrix.shape)

cosine_sim2 = cosine_similarity(count_matrix, count_matrix)
print(cosine_sim2.shape)

movies_df = movies_df.reset_index()
indices = pd.Series(movies_df.index, index=movies_df['title_x'])


(4803, 14559)
(4803, 4803)


In [ ]:
#reverse mapping of indices -> helps look up title on the basis of the ranking
indices = pd.Series(movies_df.index, index=movies_df["title_x"]).drop_duplicates()
#print(indices.head())

title_x
Avatar                                      0
Pirates of the Caribbean: At World's End    1
Spectre                                     2
The Dark Knight Rises                       3
John Carter                                 4
dtype: int64


In [ ]:
def get_recommendations(title, cosine_sim=cosine_sim2):
    idx = indices[title]
    similarity_scores = list(enumerate(cosine_sim[idx]))
    similarity_scores= sorted(similarity_scores, key=lambda x: x[1], reverse=True)
    similarity_scores= similarity_scores[1:11]
    # (a, b) where a is id of movie, b is similarity_scores

    movies_indices = [ind[0] for ind in similarity_scores]
    movies = movies_df["title_x"].iloc[movies_indices]
    return movies

In [ ]:
print("################ Content Based System #############")
print("Recommendations for The Dark Knight Rises")
print(get_recommendations("The Dark Knight Rises", cosine_sim2))
print()
print("Recommendations for Avengers")
print(get_recommendations("The Avengers", cosine_sim2))


################ Content Based System #############
Recommendations for The Dark Knight Rises
119                             Batman Begins
4638                 Amidst the Devil's Wings
65                            The Dark Knight
280                            Public Enemies
405     The Fast and the Furious: Tokyo Drift
1795                                Silverado
2060                       Out of the Furnace
2218                           Death Sentence
2609                                Dark Blue
3257                          American Psycho
Name: title_x, dtype: object

Recommendations for Avengers
193                                           After Earth
26                             Captain America: Civil War
238                          Teenage Mutant Ninja Turtles
1294                                             Serenity
1303                               Star Trek: Generations
1827    Teenage Mutant Ninja Turtles II: The Secret of...
2327                                   